In [2]:
# prueba lunar lander por humano

import gymnasium as gym

env = gym.make("LunarLander-v3", render_mode="rgb_array")

import numpy as np
import pygame
import gymnasium.utils.play

lunar_lander_keys = {
    (pygame.K_UP,): 2,
    (pygame.K_LEFT,): 1,
    (pygame.K_RIGHT,): 3,
}
gymnasium.utils.play.play(env, zoom=3, keys_to_action=lunar_lander_keys, noop=0)

## Prueba 1. AG - Evolución generacional

In [40]:
from MLP import MLP
import random


def policy (observation, model):
    s = model.forward(observation)
    action = np.argmax(s)
    return action

evaluaciones = 0

def run (model):
    #observation, info = env.reset(seed=42)
    observation, info = env.reset()
    ite = 0
    racum = 0
    while True:
        action = policy(observation, model)
        observation, reward, terminated, truncated, info = env.step(action)
        
        racum += reward

        if terminated or truncated:
            r = (racum+200) / 500
            #print(racum, r)
            return racum


def run_multiple_games(ch, model, N_games):
    global evaluaciones
    model.from_chromosome(ch)
    r = 0

    for _ in range(N_games):
        r += run(model)
        evaluaciones += 1
    
    return r/N_games, ch # devuelve el refuerzo medio
    

In [41]:
# Define operadores de números reales

rang = (-1, 1) # al no hacerlo con clases, debemos definir el rango como variable global


'''
    Devuelve un individuo seleccionado por torneo. Se selecciona el individuo de mayor fitness sobre un 
    conjunto aleatorio equiprobable T.
'''

def select(pop, T, fitness_array): 
    tournament = random.sample(range(len(pop)), T)  
    # Busca el índice del mejor individuo (mayor fitness porque es acumulativo en LunarLander)
    best_index = max(tournament, key=lambda i: fitness_array[i])  
    return pop[best_index].copy()  


def create(arquitecture, N=100): 
    pop = []

    cromosoma_length = (arquitecture[0] * arquitecture[1]) + arquitecture[1] + (arquitecture[1] * arquitecture[2]) + arquitecture[2]

    for _ in range(N):
        values = [random.uniform(-1, 1) for _ in range(cromosoma_length)]  
        pop.append(values)

   
    return pop

'''
    Función que ordena una poblacion segun el fitness.
    @return: Devuelve la poblacion ordenada por fitness, y los valores fitness de cada individuo de la poblacion.
'''
def sort_pop (pop, fitness): 
    pop_with_fitness = [(indiv, fit) for indiv, fit in zip(pop, fitness)]
    sorted_pop = sorted(pop_with_fitness, key=lambda x: x[1], reverse=True)  # Mayor fitness primero
    return [indiv for indiv, _ in sorted_pop], [fit for _, fit in sorted_pop]

'''
    Funcion que implementa el operador crossover: emparejamiento de dos individuos. pcross: probabilidad de que 
    se produzca el emparejamiento. Se implementa el emparejamiento para números reales basado en una combinacion
    de los genes de ambos padres. Los genes de cada padre tienen una mayor representacion en uno de los dos hijos.
    @return: dos hijos
'''
# def crossover (ind1, ind2, pcross, arquitecture): # devuelve el cruce (emparejamiento) de dos individuos
    
#     if (random.random() > pcross):
#         return ind1.copy(), ind2.copy()

#     beta = random.uniform(0,1)
    
#     child1_x = beta*ind1[0] + (1 - beta)*ind2[0]
#     child1_y = beta*ind1[1] + (1 - beta)*ind2[1]

#     child2_x = (1 - beta)*ind1[0] + beta*ind2[0]
#     child2_y = (1 - beta)*ind1[1] + beta*ind2[1]


#     return [child1_x, child1_y], [child2_x, child2_y]

def crossover (ind1, ind2, pcross, arquitecture): # devuelve el cruce (emparejamiento) de dos individuos
    if (random.random() > pcross):
        return ind1.copy(), ind2.copy()
    child1 = []
    child2 = []
    
    for gene1, gene2 in zip(ind1, ind2):
        beta = random.random()
        
        c1 = beta * gene1 + (1 - beta) * gene2
        c2 = (1 - beta) * gene1 + beta * gene2
        
        child1.append(c1)
        child2.append(c2)
    
    return child1, child2
    

'''
    Función que muta un individuo. pmut: probabilidad de que se produzca la mutacion
    Este tipo de mutación reemplaza uno de los valores en el individuo con un nuevo valor generado
    aleatoriamente dentro de los límites definidos (rang).
'''
def mutate(ind, pmut):
    if random.random() < pmut:
        idx = random.randint(0, len(ind) - 1)
        ind[idx] = random.uniform(rang[0], rang[1])
    return ind.copy()


'''
    Algoritmo de evolución generacional. Funcionamiento:
    1. Ordena la población inicial según su fitness
    2. Itera a través de un número dado de evaluaciones (neval)
        - Si hay elitismo, guarda el mejor individuo directamente en la nueva población
        - Realiza selección por torneo para elegir padres
        - Aplica cruce con probabilidad 'pcross' para generar descendientes
        - Aplica mutación con probabilidad 'pmut' a los descendientes
        - Llena la nueva población con los descendientes generados
    3. Ordena la nueva población y evalúa el fitness
    4. Imprime el mejor fitness cada 'trace' evaluaciones, si está habilitado
    5. Devuelve la mejor población al final de las evaluaciones
    @return: poblacion final
'''
def evolve(pop, pmut, arquitecture, neval=3500, T=2, trace=100, pcross=0.7, elitism=False):
    """
    Algoritmo evolutivo con traza basada en el número de evaluaciones.
    """
    global evaluaciones
    evaluaciones = 0

    architecture = [8, 6, 4] # salida = 4; izq, der, arriba y apagado
    model = MLP(architecture)
    
    while evaluaciones < neval:
        new_poblacion = []
        fitness_array = []
        
        # -- Se ejecuta el RUN de todos los individuos de la poblacion y nos quedamos con los fitness = rewards
        for indv in pop:
            fitness_n_games, ch = run_multiple_games(indv, model, 2)
            fitness_array.append(fitness_n_games)
        
        # Genera la nueva población
        while len(new_poblacion) < len(pop):
            parent_1 = select(pop, T, fitness_array)
            parent_2 = select(pop, T, fitness_array)

            child_1, child_2 = crossover(parent_1, parent_2, pcross, arquitecture)

            child_1 = mutate(child_1, pmut)
            child_2 = mutate(child_2, pmut)

            new_poblacion.extend([child_1, child_2])

            # Verifica el límite de evaluaciones después de cada operación relevante
            if evaluaciones >= neval:
                break

        if evaluaciones >= neval:
            break

        # Ordena la nueva población y calcula fitness
        pop, fitness = sort_pop(new_poblacion[:len(pop)], fitness_array)

        if trace > 0 and evaluaciones % trace == 0:
            print(f"Evaluaciones: {evaluaciones}, Mejor fitness: {fitness[0]}")

    print(f"Evaluaciones: {evaluaciones}, Mejor fitness: {fitness[0]}")
    return pop, fitness


### Decisiones

**Operadores básicos**:
- Selección: por torneo
- Emparejamiento: mezclado lineal 
- Mutación: por rango [vmin, vmax]

**Hiperparámetros**:
* *pmut*: 10/100
* *ngen*: 100
* T: 4
* *pcross*: 0.7
* elisitm: False

In [42]:
# crea y evoluiona
best_individuals = []
himmelblau_values = []
fitness_values = []

for i in range(1,2):
    print(f"Ejecución {i}")
    pop = create(arquitecture=[8,6,4])
    pop, fitness = evolve(pop, arquitecture=[8,6,4], pmut=10/100, neval=3500, T=4, trace=1, pcross=0.7, elitism=False)

    best_individual = pop[0]  
    fitness_value = fitness[0]

    # Almacenar resultados
    best_individuals.append(best_individual)
    fitness_values.append(fitness_value)

Ejecución 1
Evaluaciones: 200, Mejor fitness: -45.83686654596207
Evaluaciones: 400, Mejor fitness: -24.31074955721847
Evaluaciones: 600, Mejor fitness: -25.246726662274014
Evaluaciones: 800, Mejor fitness: -38.186428229145335
Evaluaciones: 1000, Mejor fitness: -38.2385507515295
Evaluaciones: 1200, Mejor fitness: -35.13904651328728
Evaluaciones: 1400, Mejor fitness: -9.847544901246337
Evaluaciones: 1600, Mejor fitness: -15.551566912593522
Evaluaciones: 1800, Mejor fitness: -42.34057955666513
Evaluaciones: 2000, Mejor fitness: -24.05826282060974
Evaluaciones: 2200, Mejor fitness: 54.99716169729059
Evaluaciones: 2400, Mejor fitness: -19.64313430415241
Evaluaciones: 2600, Mejor fitness: -67.72770453244553
Evaluaciones: 2800, Mejor fitness: -30.823735016375025
Evaluaciones: 3000, Mejor fitness: -56.90987999249397
Evaluaciones: 3200, Mejor fitness: -41.36817252072982
Evaluaciones: 3400, Mejor fitness: -31.629973718056718
Evaluaciones: 3600, Mejor fitness: -31.629973718056718


In [47]:
import time

def run_lunar_lander(model, chromosome):

    env = gym.make("LunarLander-v3", render_mode="human")  
    observation, _ = env.reset() 
    model.from_chromosome(chromosome)  
    
    total_reward = 0
    done = False

    while not done:
        env.render()  
        
        action_values = model.forward(observation)  
        action = np.argmax(action_values)  
        
        observation, reward, done, _, _ = env.step(action)  
        total_reward += reward
        
        time.sleep(0.05)  
    
    env.close()
    print(f"Total reward: {total_reward}")

arquitecture = [8, 6, 4] 
model = MLP(arquitecture)


# 🔹 Ejecutar el modelo en el entorno
run_lunar_lander(model, best_individuals[0])


Total reward: -33.465313386058796


In [ ]:
import numpy as np

fitness_mean = np.mean(fitness_values)
fitness_std = np.std(fitness_values)


best_ind_index = np.argmax(fitness_values)
best_ind = best_individuals[best_ind_index]
best_fitness = fitness_values[best_ind_index]

In [ ]:
print(f"Media de fitness: {fitness_mean}")
print(f"Desviación típica de fitness: {fitness_std}")

print ("-----")

print(f"El mejor individuo es {best_ind}")
print(f"Fitness en ese individuo: {best_fitness:.10f}")

Media de fitness: 0.9425973798932168
Desviación típica de fitness: 0.0832002599174057
-----
Media de himmelblau: 0.07014512890659286
Desviación típica de himmelblau: 0.10485796294344779
-----
El mejor individuo es [3.000423765860462, 2.0033795362969467]
Fitness en ese individuo: 0.9997702835
Himmelblau de la mejor solución: 0.0002297693


In [ ]:
# crea y evoluiona
pop = create()
pop, fitness = evolve(pop, arquitecture = [8,6,4], pmut=10/100, ngen=100, T=4, trace=25, pcross=0.7, elitism=False)

Generacion 0: mejor fitness [0.2952568997016495]
Generacion 25: mejor fitness [0.838982892859534]
Generacion 50: mejor fitness [0.838982892859524]
Generacion 75: mejor fitness [0.8389828928595396]
Generacion 100: mejor fitness [0.8389828928595421]


In [ ]:
# Mejor individuo, valor en la función y su fitness
best_individual = pop[0]
fitness_best = fitness[0]
  

print(f"El mejor individuo es {best_individual}")
print(f"Fitness de la mejor solución: {fitness_best}")


El mejor individuo es [3.531729179447823, -1.77507053290786]
Valor de la función de Himmelblau en ese individuo: 0.1919194164
Fitness de la mejor solución: 0.8389828928595421
